In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# import bert
# from bert import BertModelLayer
import functools

from tensorflow import keras
import tensorflow as tf
import os
import sentencepiece as spm

import tf_sentencepiece as tfs
import tensorflow_text as tftext
import sys
sys.path.extend(["/space/users/jgeorge/git/tensorflow_repos/models"])
sys.path.extend(["/space/users/jgeorge/git/tensorflow_repos/"])
sys.path.extend(["/space/users/jgeorge/git/tensorflow_repos/sentencepiece/compiled_proto"])
# sys.path.extend(["/space/users/jgeorge/git/tensorflow_repos/sentencepiece"])


import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub

# pylint: disable=g-import-not-at-top,redefined-outer-name,reimported
# from official.modeling import model_training_utils

# from official.nlp.modeling.models import bert_classifier, bert_pretrainer
# # from official.nlp.modeling.models.bert_classifier import BertClassifier
# # from official.nlp.modeling.models.bert_pretrainer import BertPretrainer
# # from official.nlp import bert_modeling as modeling
# # from official.nlp import bert_models
# from official.nlp import optimization
# from official.nlp.bert import common_flags
# from official.nlp.bert import input_pipeline
# from official.nlp.bert import model_saving_utils
# from official.utils.misc import distribution_utils
# from official.utils.misc import keras_utils
# from official.nlp.bert import tokenization

# from official.nlp.albert import configs as albert_configs
# from official.nlp.bert import run_classifier as run_classifier_bert

from tensorflow.python.keras.utils import tf_utils
from sklearn.model_selection import train_test_split

from bert import tokenization as bert_tokenization

In [ ]:
# import sentencepiece_model_pb2
import sentencepiece_pb2

In [3]:
!pwd

/space/users/jgeorge/git/tensorflow_experiments


In [4]:
print(tf.__version__)

2.2.0


In [5]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [6]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[4]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')]


In [7]:
##setting direcory for downloading tfhub modules
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [8]:
sentence_piece_folder="/var/extra/users/jgeorge/tf2.0/sentence_piece"

In [9]:
sentence_piece_model_path=os.path.join(sentence_piece_folder,"multilingual.proto")

In [10]:
vocab_file="/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt"

In [11]:
dish_small_spm_file='/var/extra/users/jgeorge/git/tensorflow_repos/sentencepiece/build/models/spm_dish.model'
dish_small_spm_vocab='/var/extra/users/jgeorge/git/tensorflow_repos/sentencepiece/build/models/spm_dish.vocab'

In [12]:
sentence_piece_processor = spm.SentencePieceProcessor(model_file=dish_small_spm_file)

In [13]:
bert_full_tokenizer = bert_tokenization.FullTokenizer(vocab_file,do_lower_case=False)

In [14]:
bert_full_tokenizer.tokenize("talk to an agent")

['talk', 'to', 'an', 'agent']

In [15]:
bert_full_tokenizer.tokenize("talk to aagent")

['talk', 'to', 'aa', '##gent']

In [16]:
tf.constant("talk to aagent")

<tf.Tensor: shape=(), dtype=string, numpy=b'talk to aagent'>

In [17]:
bert_full_tokenizer.tokenize("talk to aagent")

['talk', 'to', 'aa', '##gent']

In [18]:
bert_full_tokenizer.convert_tokens_to_ids(bert_full_tokenizer.tokenize("talk to an agent"))

[31311, 10114, 10151, 18980]

In [13]:
vocab = []
with open(vocab_file,'r') as inp_f:
    for word in inp_f:
        vocab.append(word.strip())
        

In [14]:
vocab

['[PAD]',
 '[unused1]',
 '[unused2]',
 '[unused3]',
 '[unused4]',
 '[unused5]',
 '[unused6]',
 '[unused7]',
 '[unused8]',
 '[unused9]',
 '[unused10]',
 '[unused11]',
 '[unused12]',
 '[unused13]',
 '[unused14]',
 '[unused15]',
 '[unused16]',
 '[unused17]',
 '[unused18]',
 '[unused19]',
 '[unused20]',
 '[unused21]',
 '[unused22]',
 '[unused23]',
 '[unused24]',
 '[unused25]',
 '[unused26]',
 '[unused27]',
 '[unused28]',
 '[unused29]',
 '[unused30]',
 '[unused31]',
 '[unused32]',
 '[unused33]',
 '[unused34]',
 '[unused35]',
 '[unused36]',
 '[unused37]',
 '[unused38]',
 '[unused39]',
 '[unused40]',
 '[unused41]',
 '[unused42]',
 '[unused43]',
 '[unused44]',
 '[unused45]',
 '[unused46]',
 '[unused47]',
 '[unused48]',
 '[unused49]',
 '[unused50]',
 '[unused51]',
 '[unused52]',
 '[unused53]',
 '[unused54]',
 '[unused55]',
 '[unused56]',
 '[unused57]',
 '[unused58]',
 '[unused59]',
 '[unused60]',
 '[unused61]',
 '[unused62]',
 '[unused63]',
 '[unused64]',
 '[unused65]',
 '[unused66]',
 '[unused

In [15]:
def tokenize_custom(text):
    output_tokens = []
    unk_token="UNK"
    for token in text.split():
        sub_tokens = []
        chars = list(token)
        is_bad = False
        start = 0
        print(len(chars))
        while start < len(chars):
            print("start ",start)
            end = len(chars)
            cur_substr=None
            while start < end:
                substr = "".join(chars[start:end])
                if start>0:
                    substr = "##"+substr
                print("substr is ",substr)
                if substr in vocab:
                    print("in vocab ",substr)
                    cur_substr = substr
                    break
                end-=1
            if cur_substr is None:
                is_bad=True
                break
            print("cur_substr end", cur_substr)
            sub_tokens.append(cur_substr)
            start = end
            print("start after update ",start, "end after update ",end)
        if is_bad:
            output_tokens.append(unk_token)
        else:
            output_tokens.extend(sub_tokens)
    return output_tokens
                        
tokenize_custom("talk to aagent")   

4
start  0
substr is  talk
in vocab  talk
cur_substr end talk
start after update  4 end after update  4
2
start  0
substr is  to
in vocab  to
cur_substr end to
start after update  2 end after update  2
6
start  0
substr is  aagent
substr is  aagen
substr is  aage
substr is  aag
substr is  aa
in vocab  aa
cur_substr end aa
start after update  2 end after update  2
start  2
substr is  ##gent
in vocab  ##gent
cur_substr end ##gent
start after update  6 end after update  6


['talk', 'to', 'aa', '##gent']

In [22]:
s = tf.Variable([''],shape=(None,))
s

<tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([b''], dtype=object)>

In [23]:
tf.concat([s,['abc', 'def']],axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'', b'abc', b'def'], dtype=object)>

In [17]:
is_bad = tf.Variable(False)
is_bad

<tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False>

In [16]:
missing_index=-1
vocab_table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(vocab_file,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),default_value=missing_index)
def is_vocab_missing(vocab_lookup):
    return vocab_lookup==missing_index

def inner_cond(start,end,word,chars,vocab_lookup):
    cond = tf.less(start,end) and is_vocab_missing(vocab_lookup)
    tf.print(f'vocab lookup inside inner_cond  start {start} end {end} condition {vocab_lookup} is_vocab_missing {is_vocab_missing(vocab_lookup)} & entire cond {cond}')
    return cond

In [18]:
def get_chars_len(chars):
    chars_len = tf.shape(chars)[0]
    # when it is ragged tensor
#     chars_len = chars.nrows()
    return chars_len

In [19]:
# def inner_while_body(start,end,word,vocab_lookup):
def inner_while_body(start,end,word,chars,vocab_lookup):
    tf.print(f'inner_while_body start {start} end {end} word {word} chars {chars}\n')
#     chars = tf.strings.bytes_split(word)
    substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
#     tf.strings.reduce_join()
    if start>0:
        substring = tf.strings.join(["##",substring])
    # assigning it rather than creating a new variable so that this is accessible to other
    # function referencing this variable
    vocab_lookup.assign(vocab_table.lookup(tf.constant(substring)))
    tf.print(f'substring {substring} vocab lookup {vocab_lookup}')
    if vocab_lookup==missing_index:
#         print(f'missing {substring}')
        tf.print(f'missing {substring}')
        # subtracting only if missing
        end.assign_sub([1])
    else:
        tf.print(f'present {substring}')
    
    return (start,end,substring,chars,vocab_lookup)

In [20]:
def outer_cond(start,end,word,chars,is_bad,sub_tokens):
    tf.print(f'outer_cond start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
    return (start < tf.shape(chars)[0]) and not is_bad

def outer_while_loop(start,end,word,chars,is_bad,sub_tokens):
    tf.print(f'outer_while_loop start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
    # vocab_lookup = vocab_table.lookup(tf.constant(substring))
    chars_len = get_chars_len(chars)
    end = tf.Variable([chars_len])
    substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
    vocab_table = tf.Variable(missing_index,shape=(),name='vocab_lookup',dtype=tf.int64)
#     vocab_lookup.assign(vocab_table.lookup(tf.constant(substring)))

    (start_1,end_1,substring_1,chars_1,vocab_lookup) = tf.while_loop(inner_cond,inner_while_body,loop_vars=[start,end,substring,chars,vocab_lookup])
    tf.print(f'****end of inner while start {start_1} end {end_1} substring {substring_1} chars {chars_1} vocab_lookup {vocab_lookup} \n\n')
    if not is_vocab_missing(vocab_lookup):
        sub_tokens_concat = tf.concat([sub_tokens,[substring_1]],axis=0)
        tf.print("sub tokens concat ",sub_tokens_concat, 'shape ',tf.shape(sub_tokens_concat))
#         sub_tokens.append(substring)
        tf.print('subtokens ',sub_tokens, ' sub tokens shape ',sub_tokens)
        sub_tokens.assign(sub_tokens_concat)
        tf.print(f'subtokens appended ')
    else:
        # if vocab missing even after the inner loop then this word can't be formed using
        # word existing in the dictionary
        is_bad.assign(True)
    tf.print(f'start before assigning end {start} & end val {end}')
    start.assign(end_1)
    tf.print(f'current subtokens {sub_tokens}')
    return (start,end,word,chars,is_bad,sub_tokens)
            
    

In [21]:
def per_word_tokenize(word):
    word = tf.squeeze(word)
    chars = tf.strings.bytes_split(word)#.to_tensor()
    tf.print(f'per_word_tokenize word  {word} shape {tf.shape(word)} chars {chars}')
#     chars_len = tf.shape(chars)[0]
    chars_len = get_chars_len(chars)
#     start = tf.zeros((1,),dtype=tf.int32)
    start = tf.Variable([0],dtype=tf.int32)
    end = tf.Variable([chars_len])

    missing_index=-1
    
    is_bad = tf.Variable(False)
    sub_tokens = tf.Variable([],shape=(None,),dtype=tf.string)
    
    (start,end,word,chars,is_bad,sub_tokens_1) =  tf.while_loop(outer_cond,outer_while_loop,loop_vars=[start,end,word,chars,is_bad,sub_tokens])
    sub_tokens.assign(sub_tokens_1)
#     sub_tokens_out = tf.expand_dims(sub_tokens,axis=0)
    return sub_tokens

In [22]:
sub_tokens = per_word_tokenize('aagent')

per_word_tokenize word  b'aagent' shape [] chars [b'a' b'a' b'g' b'e' b'n' b't']
outer_cond start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([6], dtype=int32)>  chars [b'a' b'a' b'g' b'e' b'n' b't'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
outer_while_loop start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([6], dtype=int32)>  chars [b'a' b'a' b'g' b'e' b'n' b't'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>


UnboundLocalError: local variable 'vocab_lookup' referenced before assignment

In [31]:
sub_tokens

<tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([b'aa', b'##gent'], dtype=object)>

In [36]:
tf.strings.reduce_join(sub_tokens,separator=' ')

<tf.Tensor: shape=(), dtype=string, numpy=b'aa ##gent'>

In [32]:
def joined_per_word_tokenized(word):
    sub_tokens = per_word_tokenize(word)
    return tf.strings.reduce_join(sub_tokens,separator=' ')

In [33]:
per_word_tokenize('aagent')

per_word_tokenize word  b'aagent' shape [] chars [b'a' b'a' b'g' b'e' b'n' b't']
outer_cond start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([6], dtype=int32)>  chars [b'a' b'a' b'g' b'e' b'n' b't'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
outer_while_loop start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([6], dtype=int32)>  chars [b'a' b'a' b'g' b'e' b'n' b't'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
vocab lookup inside inner_cond  start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variab

<tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([b'aa', b'##gent'], dtype=object)>

In [368]:
s1 = tf.constant('agent aagent')
slist = tf.strings.split(s1)
slist2 = tf.strings.bytes_split(slist)
slist2

<tf.RaggedTensor [[b'a', b'g', b'e', b'n', b't'], [b'a', b'a', b'g', b'e', b'n', b't']]>

In [34]:
tf.strings.split('talk to an agent, talk to an aagent',sep=',')

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'talk to an agent', b' talk to an aagent'], dtype=object)>

In [73]:
def process_sentence(sentence,max_sequence_length=10):
    words = tf.strings.split(sentence)
    tf.print(f'words in process sentence {words}')
#     list_words = tf.expand_dims(words,axis=1)
#     tf.print(f'words in process sentence {list_words} shape {tf.shape(list_words)}')
    subword_tokenized_list = tf.map_fn(joined_per_word_tokenized,words,parallel_iterations=None,infer_shape=False)
    sentence_tokenized = tf.strings.reduce_join(subword_tokenized_list,separator=' ')
    processed_tokens = tf.strings.split(sentence_tokenized)
    tf.print(f'processed tokens {processed_tokens}')
    sequence_length = tf.shape(processed_tokens)[-1]
    trimmed_max_length = max_sequence_length-2
    values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                    lambda : tf.slice(processed_tokens,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : processed_tokens)
    concat = tf.concat([['[CLS]'],values_trimmed,['[SEP]']],axis=0,name='concat_out')
    actual_token_length = tf.shape(concat)[-1]
    token_ids = vocab_table.lookup(concat)
    padded = tf.pad(token_ids,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
    return padded
#     return sentence_tokenized
#     return subword_tokenized_list
#     return processed_tokens

In [67]:
process_sentence('talk to an aagent')

words in process sentence [b'talk' b'to' b'an' b'aagent']
per_word_tokenize word  b'talk' shape [] chars [b't' b'a' b'l' b'k']
outer_cond start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
outer_while_loop start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
vocab lookup inside inner_cond  start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)

<tf.Tensor: shape=(10,), dtype=int64, numpy=
array([  101, 31311, 10114, 10151, 28335, 22500,   102,     0,     0,
           0])>

In [36]:
tf.strings.split()

AttributeError: 'DType' object has no attribute 'split'

In [75]:
def process_sentences(sentences):
    sentences_tokenized = tf.map_fn(process_sentence,sentences,dtype=tf.int64)
    return sentences_tokenized

In [76]:
sentence_list = tf.strings.split('talk to an agent, talk to an aagent',sep=',')
print(sentence_list)
process_sentences(sentence_list)

tf.Tensor([b'talk to an agent' b' talk to an aagent'], shape=(2,), dtype=string)
words in process sentence [b'talk' b'to' b'an' b'agent']
per_word_tokenize word  b'talk' shape [] chars [b't' b'a' b'l' b'k']
outer_cond start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
outer_while_loop start <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([0], dtype=int32)> end <tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([4], dtype=int32)>  chars [b't' b'a' b'l' b'k'] is_bad <tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False> sub_tokens <tf.Variable 'Variable:0' shape=(None,) dtype=string, numpy=array([], dtype=object)>
vocab lookup inside inner_cond  start

<tf.Tensor: shape=(2, 10), dtype=int64, numpy=
array([[  101, 31311, 10114, 10151, 18980,   102,     0,     0,     0,
            0],
       [  101, 31311, 10114, 10151, 28335, 22500,   102,     0,     0,
            0]])>

In [ ]:
tf.keras.layers.Layer()

In [82]:
class BertTokenizationCustom(tf.keras.layers.Layer):
    def __init__(self,vocab_path):
        super(BertTokenizationCustom, self).__init__(trainable=False,dtype=tf.int32)
        
        self.missing_index=-1
        self.vocab_path = vocab_path
        self.vocab_table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(vocab_file,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),default_value=missing_index)
    def build(self,input_shape):
        #self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        ## pad_id is usually 0 
#         [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','[PAD]'],model_file=self.model_path)
        self.built=True
        
#     @tf.function    
    def call(self, input_text):
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
        encoded_text = self.process_sentences(input_text)
        return encoded_text
    
    def is_vocab_missing(self,vocab_lookup):
        return vocab_lookup==self.missing_index
    
    def get_chars_len(self,chars):
        chars_len = tf.shape(chars)[0]
        # when it is ragged tensor
    #     chars_len = chars.nrows()
        return chars_len

    def inner_cond(self,start,end,word,chars,vocab_lookup):
        cond = tf.less(start,end) and is_vocab_missing(vocab_lookup)
#         tf.print(f'vocab lookup inside inner_cond  start {start} end {end} condition {vocab_lookup} is_vocab_missing {is_vocab_missing(vocab_lookup)} & entire cond {cond}')
        return cond
    
    # def inner_while_body(start,end,word,vocab_lookup):
    def inner_while_body(self,start,end,word,chars,vocab_lookup):
#         tf.print(f'inner_while_body start {start} end {end} word {word} chars {chars}\n')
    #     chars = tf.strings.bytes_split(word)
        substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
    #     tf.strings.reduce_join()
        if start>0:
            substring = tf.strings.join(["##",substring])
        # assigning it rather than creating a new variable so that this is accessible to other
        # function referencing this variable
        vocab_lookup.assign(self.vocab_table.lookup(tf.constant(substring)))
#         tf.print(f'substring {substring} vocab lookup {vocab_lookup}')
        if vocab_lookup==missing_index:
    #         print(f'missing {substring}')
#             tf.print(f'missing {substring}')
            # subtracting only if missing
            end.assign_sub([1])
#         else:
#             tf.print(f'present {substring}')

        return (start,end,substring,chars,vocab_lookup)
    
    def outer_cond(self,start,end,word,chars,is_bad,sub_tokens):
#         tf.print(f'outer_cond start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
        return (start < tf.shape(chars)[0]) and not is_bad

    def outer_while_loop(self,start,end,word,chars,is_bad,sub_tokens):
#         tf.print(f'outer_while_loop start {start} end {end}  chars {chars} is_bad {is_bad} sub_tokens {sub_tokens}')
        # vocab_lookup = vocab_table.lookup(tf.constant(substring))
        chars_len = get_chars_len(chars)
        end = tf.Variable([chars_len])
        substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
        vocab_lookup = tf.Variable(missing_index,shape=(),name='vocab_lookup',dtype=tf.int64)
    #     vocab_lookup.assign(vocab_table.lookup(tf.constant(substring)))

        (start_1,end_1,substring_1,chars_1,vocab_lookup) = tf.while_loop(inner_cond,inner_while_body,loop_vars=[start,end,substring,chars,vocab_lookup])
#         tf.print(f'****end of inner while start {start_1} end {end_1} substring {substring_1} chars {chars_1} vocab_lookup {vocab_lookup} \n\n')
        if not is_vocab_missing(vocab_lookup):
            sub_tokens_concat = tf.concat([sub_tokens,[substring_1]],axis=0)
#             tf.print("sub tokens concat ",sub_tokens_concat, 'shape ',tf.shape(sub_tokens_concat))
    #         sub_tokens.append(substring)
#             tf.print('subtokens ',sub_tokens, ' sub tokens shape ',sub_tokens)
            sub_tokens.assign(sub_tokens_concat)
#             tf.print(f'subtokens appended ')
        else:
            # if vocab missing even after the inner loop then this word can't be formed using
            # word existing in the dictionary
            is_bad.assign(True)
#         tf.print(f'start before assigning end {start} & end val {end}')
        start.assign(end_1)
#         tf.print(f'current subtokens {sub_tokens}')
        return (start,end,word,chars,is_bad,sub_tokens)
    
    def per_word_tokenize(self,word):
        word = tf.squeeze(word)
        chars = tf.strings.bytes_split(word)#.to_tensor()
#         tf.print(f'per_word_tokenize word  {word} shape {tf.shape(word)} chars {chars}')
    #     chars_len = tf.shape(chars)[0]
        chars_len = get_chars_len(chars)
    #     start = tf.zeros((1,),dtype=tf.int32)
        start = tf.Variable([0],dtype=tf.int32)
        end = tf.Variable([chars_len])

        missing_index=-1

        is_bad = tf.Variable(False)
        sub_tokens = tf.Variable([],shape=(None,),dtype=tf.string)

        (start,end,word,chars,is_bad,sub_tokens_1) =  tf.while_loop(outer_cond,outer_while_loop,loop_vars=[start,end,word,chars,is_bad,sub_tokens])
        sub_tokens.assign(sub_tokens_1)
    #     sub_tokens_out = tf.expand_dims(sub_tokens,axis=0)
        return sub_tokens
    def joined_per_word_tokenized(self,word):
        sub_tokens = per_word_tokenize(word)
        return tf.strings.reduce_join(sub_tokens,separator=' ')
    
    def process_sentence(self,sentence,max_sequence_length=10):
        words = tf.strings.split(sentence)
#         tf.print(f'words in process sentence {words}')
    #     list_words = tf.expand_dims(words,axis=1)
    #     tf.print(f'words in process sentence {list_words} shape {tf.shape(list_words)}')
        subword_tokenized_list = tf.map_fn(joined_per_word_tokenized,words,parallel_iterations=None,infer_shape=False)
        sentence_tokenized = tf.strings.reduce_join(subword_tokenized_list,separator=' ')
        processed_tokens = tf.strings.split(sentence_tokenized)
#         tf.print(f'processed tokens {processed_tokens}')
        sequence_length = tf.shape(processed_tokens)[-1]
        trimmed_max_length = max_sequence_length-2
        values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                        lambda : tf.slice(processed_tokens,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : processed_tokens)
        concat = tf.concat([['[CLS]'],values_trimmed,['[SEP]']],axis=0,name='concat_out')
        actual_token_length = tf.shape(concat)[-1]
        token_ids = self.vocab_table.lookup(concat)
        padded = tf.pad(token_ids,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
        return padded
    
    def process_sentences(self,sentences):
        sentences_tokenized = tf.map_fn(process_sentence,sentences)
        return sentences_tokenized


In [72]:
tf.while_loop(inner_cond,inner_while_body,loop_vars=[start,end,word,vocab_lookup])

vocab lookup inside condition <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1> True & entire cond True
vocab lookup <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1>
missing b'aagent'
vocab lookup inside condition <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1> True & entire cond True
vocab lookup <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1>
missing b'aagen'
vocab lookup inside condition <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1> True & entire cond True
vocab lookup <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1>
missing b'aage'
vocab lookup inside condition <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1> True & entire cond True
vocab lookup <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1>
missing b'aag'
vocab lookup inside condition <tf.Variable 'vocab_lookup:0' shape=() dtype=int64, numpy=-1> True & entire cond True
vocab lookup <tf.Variable 'vocab_lookup:0' shape=() dt

(<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'aa'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=28335>)

In [84]:
vocab_file

'/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt'

In [83]:
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
tokenized_out = BertTokenizationCustom(vocab_path=vocab_file)(input_text)
model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=tokenized_out)

TypeError: in user code:

    <ipython-input-79-fe51f0aa31a9>:20 call  *
        encoded_text = self.process_sentences(input_text)
    <ipython-input-82-4978118e6274>:133 process_sentences  *
        sentences_tokenized = tf.map_fn(process_sentence,sentences)
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:574 new_func  **
        return func(*args, **kwargs)
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow/python/ops/map_fn.py:425 map_fn_v2
        name=name)
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow/python/ops/map_fn.py:270 map_fn
        maximum_iterations=n)
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:2688 while_loop
        back_prop=back_prop)
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow/python/ops/while_v2.py:196 while_loop
        add_co

In [22]:
lookup_out = VocabLookup(vocab_path=vocab_path)(input_text)
lookup_out2 = VocabLookup2()(input_text)

In [23]:
model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out)
model_lookup2 = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out2)

In [55]:
substring = tf.strings.join(tf.slice(chars,begin=start,size=end-start))
substring

<tf.Tensor: shape=(), dtype=string, numpy=b'aagen'>

In [46]:
chars[0:5]

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'a', b'a', b'g', b'e', b'n'], dtype=object)>

In [ ]:
# tensor grgaph for Innermost while loop:
# while start < end:
#     substr = "".join(chars[start:end])
#     if start>0:
#         substr = "##"+substr
#     if substr in vocab:
#         cur_substr = substr
#         break
#     end-=1

input_text= tf.strings.split('talk to an agent')
CharList = tf.strings.bytes_split(aa)
CharListLen = tf.strings.length(aa)
i =tf.constant(0)
end = CharListLen[i].numpy()
input_word =input_text[i].numpy()
#input_word = CharList[i].numpy()
substr =tf.Variable('',tf.string)
def innerWhileCond(start,end,word):
    return tf.less(start, end)
    
def innerWhileBody(start,end,word):
    preString = tf.Variable("##",tf.string)
    chars = tf.strings.bytes_split(word)
    substr = tf.strings.join(chars[start:end])
    def f1(preString,substr):
        substr = tf.strings.join(preString,substr)
        return substr
    def f3(substr):
        substr = substr
        return substr
    substr= tf.cond(tf.less(0, start), f1(preString,substr),f3(substr))
    tf.subtract(end,1)
#     def f2():
#         curr_substr = substr
#         return curr_substr
#     def f3():
#         return tf.subtract(end,1)
#     tf.cond(tf.less(0, start), f2, f3)
    return (start,end,substr)


res = tf.while_loop(innerWhileCond,innerWhileBody,[i,end,input_word])
print(res)
    
    
    
    
    


In [ ]:
table_init = tf.lookup.TextFileInitializer(self.vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True

In [18]:
class FullTokenizer(tf.keras.layers.Layer):
    def __init__(self,vocab_path):
        super(FullTokenizer, self).__init__(trainable=False,dtype=tf.int32)
        self.vocab_path = vocab_path
    def build(self,input_shape):
        table_init = tf.lookup.TextFileInitializer(self.vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text)
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup, self).get_config()
        config.update({'vocab_path': self.vocab_path})
        return config

In [19]:
input_text="talk to an agent"
tf.strings.split(input_text)

<tf.Tensor: shape=(4,), dtype=string, numpy=array([b'talk', b'to', b'an', b'agent'], dtype=object)>

In [43]:
tf.keras.backend.clear_session()
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
tokenizer_out = FullTokenizer(vocab_path=vocab_file)(input_text)
model_tokenizer = tf.keras.Model(inputs={'input_text':input_text},outputs=tokenizer_out)

AttributeError: in converted code:

    <ipython-input-41-211a43b9ec10>:12 call  *
        splitted_text = tf.strings.split(input_text).to_tensor()

    AttributeError: 'SparseTensor' object has no attribute 'to_tensor'


In [ ]:
print('predict from model1 ', model_tokenizer.predict(['hi testing lookup in tf randomtext','testing lookup']))

In [ ]:
model_lookup.save(model_dir_lookup)
model_lookup_loaded = tf.keras.models.load_model(model_dir_lookup)
print('loaded model config 1 - \n',model_lookup_loaded.get_config(),'\n')
print('predict from loaded model1 ',model_lookup_loaded.predict(['hi testing lookup in tf randomtext']))

In [ ]:
# https://github.com/tensorflow/tensorflow/issues/38305
class VocabLookup(tf.keras.layers.Layer):
    def __init__(self,vocab_path):
        super(VocabLookup, self).__init__(trainable=False,dtype=tf.int32)
        self.vocab_path = vocab_path
    def build(self,input_shape):
        table_init = tf.lookup.TextFileInitializer(self.vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text).to_tensor()
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup, self).get_config()
        config.update({'vocab_path': self.vocab_path})
        return config


In [33]:
spt = sentencepiece_pb2.SentencePieceText()

In [38]:
spt.ParseFromString(sentence_piece_processor.encode_as_serialized_proto('talk to an agent'))


170

In [39]:
print(spt)

text: "talk to an agent"
pieces {
  piece: "\342\226\201t"
  id: 3
  surface: "t"
  begin: 0
  end: 1
}
pieces {
  piece: "a"
  id: 135
  surface: "a"
  begin: 1
  end: 2
}
pieces {
  piece: "l"
  id: 140
  surface: "l"
  begin: 2
  end: 3
}
pieces {
  piece: "k"
  id: 151
  surface: "k"
  begin: 3
  end: 4
}
pieces {
  piece: "\342\226\201to"
  id: 10
  surface: " to"
  begin: 4
  end: 7
}
pieces {
  piece: "\342\226\201an"
  id: 43
  surface: " an"
  begin: 7
  end: 10
}
pieces {
  piece: "\342\226\201a"
  id: 9
  surface: " a"
  begin: 10
  end: 12
}
pieces {
  piece: "ge"
  id: 27
  surface: "ge"
  begin: 12
  end: 14
}
pieces {
  piece: "nt"
  id: 95
  surface: "nt"
  begin: 14
  end: 16
}



In [57]:
# sentence_piece_processor.load_vocabulary(dish_small_spm_vocab,1)
sentence_piece_processor.load_vocabulary(vocab_file,1)

True

In [59]:
sentence_piece_processor.reset_vocabulary()

True

In [60]:

sentence_piece_processor.encode_as_pieces("talk to an agent")

['▁t', 'a', 'l', 'k', '▁to', '▁an', '▁a', 'ge', 'nt']

In [ ]:
sentence_piece_processor.tokenize("talk to an agent")

In [68]:
sentence_piece_processor.decode_ids([3, 4, 12, 13, 191, 3, 4, 14, 3, 85, 3, 12, 117, 19, 4])

'talk to an agent'

In [22]:
vocab_file

'/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt'

In [25]:
sentence_piece_processor.set_vocabulary(vocab_file)

TypeError: not a list

In [20]:
sentence_piece_processor.load_vocabulary(vocab_file,75)

RuntimeError: Internal: /sentencepiece/src/sentencepiece_processor.cc(137) [model_] Model is not initialized.